In [1]:
using StochasticPrograms
using LinearAlgebra
using Statistics
using HiGHS
using SparseArrays
using GLPK
using JuMP

In [2]:
demands1 = [150 160 170]
demands2 = [100 120 135]
demands3 = [250 270 300]
demands4 = [300 325 350]
demands5 = [600 700 800]

probs1 = [0.25 0.5 0.25]
probs2 = [0.25 0.5 0.25]
probs3 = [0.25 0.5 0.25]
probs4 = [0.3 0.4 0.3]
probs5 = [0.3 0.4 0.3]

scenarios = Iterators.product(demands1,demands2,demands3,demands4,demands5) |> collect
prob = Iterators.product(probs1, probs2, probs3, probs4, probs5) |> collect

factories = 3
markets = 5
trans_cost = [2.49 5.21 3.76 4.85 2.07; 1.46 2.54 1.83 1.86 4.76; 3.26 3.08 2.60 3.76 4.45]
prod_cost = 14
total_cost = trans_cost .+ prod_cost
price = 24
waste_cost = 4

cap = [500 450 650]
#solver = HiGHS.Optimizer
solver = GLPK.Optimizer

GLPK.Optimizer

In [67]:
function firststage(a_v)
    m = Model(solver)

    @variable(m, Ship[i in 1:factories, j in 1:markets] >= 0)
    @variable(m, θ[i in 1:243])
    @constraint(m, [i in 1:factories], sum(Ship[i,j] for j in 1:markets) <= cap[i])
    @objective(m, Min,
        sum(total_cost[i,j]*Ship[i,j] for i in 1:factories, j in 1:markets)
        + 0.5 * sum((Ship[i,j] - a_v[i,j])^2 for i in 1:factories, j in 1:markets)
    )
    
    return m, Ship, θ, a_v
end

firststage (generic function with 1 method)

In [69]:
function master_objective(m::Model, Ship, θ, a_v)
    @objective(m, Min,
        sum(total_cost[i,j]*Ship[i,j] for i in 1:factories, j in 1:markets)
        + 0.5 * sum((Ship[i,j] - a_v[i,j])^2 for i in 1:factories, j in 1:markets)
        + sum(θ[i] for i in 1:243)
    )
    return m
end

master_objective (generic function with 1 method)

In [71]:
function secondstageCore(Ship, demands)
    m = Model()

    @variable(m, Sales[j in 1:markets] >= 0)
    @variable(m, Waste[j in 1:markets] >= 0)
    
    unit_mat = sparse([1,2,3,4,5],[1,2,3,4,5],[-1,-1,-1,-1,-1])
    T = [unit_mat unit_mat unit_mat; spzeros(5,15)]
    
    h = sparse([zeros(5); demands])


    recourseConstraints = []

    return m, Sales, Waste, recourseConstraints, h, T
end 

secondstageCore (generic function with 1 method)

In [73]:
function secondstage(Ship, demands)
    m, Sales, Waste, recourseConstraints, h, T = secondstageCore(Ship, demands)



    for j = 1:markets    
        push!(recourseConstraints, @constraint(m, Waste[j] + Sales[j] == sum(Ship[i,j] for i in 1:factories)))
    end

    for j = 1:markets
        push!(recourseConstraints, @constraint(m, Sales[j] <= demands[j]))
    end
    @objective(m, Min, -sum(price * Sales[j] for j in 1:markets)
                           + sum(waste_cost * Waste[j] for j in 1:markets))

    set_optimizer(m, solver)
    optimize!(m)
    
    return m, recourseConstraints, h, T
end

secondstage (generic function with 1 method)

In [75]:
function secondstagefeasibility(Ship, demands)
    m, Sales, Waste, recourseConstraints, h, T = secondstageCore(Ship, demands)

    nb = 10
    @variable(m, w[1:nb] >= 0)
    for j = 1:markets    
        push!(recourseConstraints, @constraint(m, Waste[j] + Sales[j] + w[j] == sum(Ship[i,j] for i in 1:factories)))
    end

    for j = 1:markets
        push!(recourseConstraints, @constraint(m, Sales[j] - w[(j+markets)] <= demands[j]))
    end

    @objective(m, Min, sum(w[i] for i in 1:nb))

    set_optimizer(m, solver)
    optimize!(m)
    σ = dual.(recourseConstraints)  # since the objective function is a minimization, we have the correct sign.

    return σ, h, T
end

secondstagefeasibility (generic function with 1 method)

In [77]:
function checkQ(expected_Q, θ, tol = 1e-10)
    a = 0
    for i in 1:243 
        if ((θ[i] > expected_Q[i])|| isapprox(θ[i]-expected_Q[i], 0.0; atol=tol, rtol=0))
            a += 1
        end
    end
    return (a >= 243)
end

checkQ (generic function with 2 methods)

In [79]:
function stop(total_cost, xsol, a_v, etheta, Qa, tol = 1e-10)
    factories = 3
    markets = 5
    Left = sum(total_cost[i,j]*xsol[i,j] for i in 1:factories, j in 1:markets) + etheta
    Right = sum(total_cost[i,j]*a_v[i,j] for i in 1:factories, j in 1:markets) + Qa
    println(Left)
    println(Right)
    println("--------------------------------------------------------------------------------------")
    return (isapprox(Left-Right, 0.0, atol=tol, rtol=0))
end

stop (generic function with 2 methods)

In [81]:
function lshaped_multicut_regularized(scenarios, prob)
    nScenarios = 243
    val = 10
    a1 = [val val val val val]
    a_v = [a1 ; a1 ; a1]
    xsol = a_v
    k = 0       # iteration index
    nfcuts = 0  # number of feasibility cuts
    nocuts = 0  # number of optimality cuts
    
    first, Ship, θ = firststage(a_v)
    n = 15
    Q_list = [+Inf for i in 1:243]
    valθ = [-Inf for i in 1:243]
    etheta = -Inf
    total_Q = 0 
    while (k <= 100)
        k += 1
        #println(first)
        optimize!(first)
        status = termination_status(first)
        
        if (status != MOI.OPTIMAL)
            println("Error: status ", status)
            return status, Ship, first
        end
        
        # status == MOI.OPTIMAL
        xsol = value.(Ship)
        if k!=1
            valθ = value.(θ)
        end

        
        E = zeros(n)'
        e = 0.0
        Q = 0.0 

        etheta = 0
        total_Q = 0
        total_Q_a = 0
        # Solve the second-stage programs
        # step 3
        for iter = 1:243
            p = prod(prob[iter])
            m, scstrs, h, T = secondstage(xsol, collect(scenarios[iter]))
            m_a, scstrs_a, h_a, T_a = secondstage(a_v, collect(scenarios[iter]))
            status = termination_status(m)
            if (status == MOI.INFEASIBLE)
                if dual_status(m) == MOI.INFEASIBILITY_CERTIFICATE
                    # If the solver emits a infeasibility certificate, we can rely on it
                    # Note: this has been tested with HiGHS only
                    # Get dual ray from constraints (infeasibility certificate):
                    # For each constraint, try to extract its dual multiplier

                    # We cannot use all_constraints as JuMP reorganize the constraints by blocks of the same type
                    # σ = dual.(all_constraints(m; include_variable_in_set_constraints=false))
                    σ = dual.(scstrs)

                    # The sign of the dual variables depend of the sense of the optimization (min or max)
                    # Julia uses the conic duality (see https://jump.dev/MathOptInterface.jl/stable/background/duality/)
                    # As a consequence, we have to take the opposite of the dual variables in case of a minimization
                    sense = objective_sense(m)
                    if (sense == MOI.MAX_SENSE)
                        σ *= -1
                    end
                else
                    # No infeasibility certificate available
                    # We explictly solve the feasibility second-stage problem
                    σ, h, T = secondstagefeasibility(xsol, collect(scenarios[i]))
                end

                # Build a feasibility cut
                E = σ'*T
                @constraint(first, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) >= σ'*h)
                nfcuts += 1
                break;
            elseif (status == MOI.OPTIMAL)
                # Step 3 part 2 + Step 4
                # Build the optimality cut component
                π = dual.(scstrs)
                E = p*(π'*T)
                e = p*(dot(π,h))
                #Q = e-sum(E[(5*(i-1)+j)]*xsol[i,j] for i in 1:factories, j in 1:markets)
                Q = p*objective_value(m)
                Q_a = p*objective_value(m_a)
                
                total_Q += Q
                total_Q_a += Q_a
                etheta += e*valθ[iter]
                if (nocuts == 0)
                    # add θ in the problem
                    @constraint(first, con, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) + θ[iter] >= e)
                    #master_objective(first, Ship, θ, a_v)
                else
                    check = valθ[iter]
                    if (check < Q)
                        @constraint(first, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) + θ[iter] >= e)
                    else
                        a_v = xsol
                    end
                end
                nocuts += 1        
            else
                println("Error second-stage resolution: status ", status)
                return status, Ship, first
            end
        end
        # Step 5
        Left = dot(total_cost,xsol) + total_Q
        Right = dot(total_cost,a_v) + total_Q_a
        if Left < Right || isapprox(Left-Right, 0.0; atol=eps(Float64), rtol=0)
            a_v = xsol
        end
        master_objective(first, Ship, θ, a_v)

    end
    optimize!(first)
    return k,nfcuts,nocuts,Ship, first, a_v
end

lshaped_multicut_regularized (generic function with 1 method)

In [59]:
k,nfcuts,nocuts,Ship, firstst, aa_v = lshaped_multicut_regularized(scenarios, prob)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Iteration, Runtime, ObjVal, NullspaceDim
0, 0.000300, 750.000000, 243
2, 0.002184, 750.000000, 243
Model   status      : Optimal
QP ASM    iterations: 2
Objective value     :  7.5000000000e+02
HiGHS run time      :          0.00
Iteration, Runtime, ObjVal, NullspaceDim
0, 0.003405, 750.000000, 0
31, 0.004128, -1377.476300, 15
Model   status      : Optimal
QP ASM    iterations: 31
Objective value     : -1.3774763000e+03
HiGHS run time      :          0.00
Iteration, Runtime, ObjVal, NullspaceDim
0, 0.006090, 2127.476300, 0
31, 0.006875, -2092.028900, 15
Model   status      : Optimal
QP ASM    iterations: 31
Objective value     : -2.0920289000e+03
HiGHS run time      :          0.01
Iteration, Runtime, ObjVal, NullspaceDim
0, 0.009700, 4219.505200, 0
31, 0.010551, -2806.581500, 15
Model   status      : Optimal
QP ASM    iterations: 31
Objective value     : -2.8065815000e+03
HiGHS 

(101, 0, 24543, VariableRef[Ship[1,1] Ship[1,2] … Ship[1,4] Ship[1,5]; Ship[2,1] Ship[2,2] … Ship[2,4] Ship[2,5]; Ship[3,1] Ship[3,2] … Ship[3,4] Ship[3,5]], A JuMP Model
Minimization problem with:
Variables: 258
Objective function type: QuadExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 11207 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: Ship, con, θ, [77.23441448801717 0.0 … 0.0 401.58829818665095; 82.53117102396529 28.14837826797388 … 196.14837826797387 34.86450435729835; 0.23441448801757137 71.85162173202616 … 103.85162173202626 163.54719745605064])

In [61]:
println("Solved in ", k, " iterations.")
println("Solved using ", nfcuts, " nfcuts.")
println("Solved using ", nocuts, " nocuts.")

Solved in 101 iterations.
Solved using 0 nfcuts.
Solved using 24543 nocuts.


In [63]:
objective_value(firstst)

-10365.791068366205

In [65]:
value.(Ship)

3×5 Matrix{Float64}:
 77.1559   0.0      12.2508    0.0    403.491
 82.8441  28.0991  108.839   196.779   33.4388
  0.0     71.9009  129.411   103.221  163.07

In [49]:
value.(aa_v)

3×5 Matrix{Float64}:
 52.4895   3.89355   69.56    75.1202  150.67
 69.1789  51.7429   103.349  129.05     96.6793
 37.8595  44.3636    91.6     95.8302  105.45